In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")


In [2]:
# Creamos un nuevo chat
import pyperclip
import uuid
from src.db.mongo.chats_manager import create_new_chat, find_chat_by_id

# user_id = uuid.uuid4()
user_id = uuid.UUID("a447f16a-2fa4-44c7-9f71-0c833b77628e")
chat_id = create_new_chat(user_id)
pyperclip.copy(str(chat_id))
print(chat_id)

Nuevo chat creado!
Conexion finalizada
aed323cd-b7ce-48f3-9d52-dcc27145161a


In [3]:
# Buscamos el chat en la db
import uuid
from src.db.mongo.chats_manager import find_chat_by_id

chat_id = uuid.UUID("3fed4781-755e-4dc5-8621-3922ac70b084")
chat_document = find_chat_by_id(chat_id)

chat_document_copy = chat_document.copy()

Chat has been found
Conexion finalizada


In [4]:
# Usamos la funcion chat

from src.app.app import chat

new_user_message = "i need to find the max static pressure for EMED-3135.01-067"
llm_collector, collector, res = chat(
    new_user_message=new_user_message,
    chat_document=chat_document_copy,
)


generate-request
{'text': '\nsummary:  The user asks the assistant to find the max static pressure for EMED-3135.01-067.\nuser_intent:  The user wants to find the max static pressure for a specific product EMED-3135.01-067.\nslots:  product=EMED-3135.01-067'} 


enhanced-request
{'text': '\nresponse:  The user is looking for the maximum static pressure of the specific product EMED-3135.01-067.'} 


request-type
{'text': "\ntype:  complex\nanalysis: This input is out of my knowledge, the user is asking for specific information about a product and its measurement system, which I don't have access to."} 


complex_request_process_modification
enhanced-request
{'text': '\nresponse:  The user is looking for the maximum static pressure for the product EMED-3135.01-067.'} 


technical-terms
{'text': '\nterms: static pressure, product, EMED-3135.01-067'} 


flavored-keywords
{'text': '\nresponse: variable, measurement system, firmware, flow computer'} 


complex_request_process_semantics
seman

C:\Users\lauth\OneDrive\Desktop\sql_assistant_v3\src\utils\sql_utils.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


summary-response-sql
{'text': 'response: The maximum static pressure for the product EMED-3135.01-067 is 61.4377.'} 


post-process-translation
{'text': 'detected_language: English.  \nresponse: The maximum static pressure for the product EMED-3135.01-067 is 61.4377.'} 




In [11]:
collector.semantic_info.keys()

dict_keys(['pla_plataforma', 'med_sistema_medicion', 'var_tipo_variable', 'var_variable_datos', 'equ_equipo', 'med_tag'])

In [13]:
print(llm_collector.llm_responses[7].llm_response)

sql_query: SELECT MAX(VD.Valor) AS 'Max Static Pressure' FROM dbo_v2.var_variable_datos AS VD INNER JOIN dbo_v2.med_sistema_medicion AS MSM ON VD.idSistemaMedicion_fk = MSM.Id INNER JOIN dbo_v2.var_tipo_variable AS VTV ON VD.IdVariable_fk = VTV.Id WHERE MSM.Tag = 'EMED-3135.01-067' AND VTV.Nombre = 'Pressão Estática (kPa)' AND VD.Fecha BETWEEN '2023-01-01' AND '2023-12-31'
suggestion: To improve the query results, specify a date range or a specific date for the measurement if applicable, as the current query does not limit the date range for the maximum static pressure.
used_tables: [var_variable_datos, med_sistema_medicion, var_tipo_variable].
table_schema: dbo_v2


In [18]:
import pandas as pd
import pyperclip
pyperclip.copy(pd.DataFrame(collector.dataframe_response).to_markdown())
print(pd.DataFrame(collector.dataframe_response).to_markdown())


|    |   Max Static Pressure |
|---:|----------------------:|
|  0 |               61.4377 |


In [26]:
instr = """According to this user request:
<Request>
The user is looking for the maximum static pressure for the product EMED-3135.01-067.
</Request>

This is an sql code:
<Sql>
SELECT MAX(VD.Valor) AS 'Max Static Pressure' FROM dbo_v2.var_variable_datos AS VD INNER JOIN dbo_v2.med_sistema_medicion AS MSM ON VD.idSistemaMedicion_fk = MSM.Id INNER JOIN dbo_v2.var_tipo_variable AS VTV ON VD.IdVariable_fk = VTV.Id WHERE MSM.Tag = 'EMED-3135.01-067' AND VTV.Nombre = 'Pressão Estática (kPa)' AND VD.Fecha BETWEEN '2023-01-01' AND '2023-12-31'
</Sql>

And this answer from database:
<dataframe>
|    |   Max Static Pressure |
|---:|----------------------:|
|  0 |               61.4377 |
</dataframe>

As an expert in SQL, your task is to understand the previous given information and explain briefly the response obtained for the user.

Note: 
User is not an expert, so he does not need a lot of technical sql details.
But you can explain understandable filters like dates or status.

Output format response:
The output should be formatted with the key format below. Do not add anything beyond the key format.
Start Key format:
"response" is the key and its content is: Brief analysis for normal user.
End of Key format

Begin!"""

suffix = "response:"

from src.components.models.llms.llms import HF_Llama38b_LLM, Openai_LLM
llama3_llm = HF_Llama38b_LLM()
llama3_llm.init_model()

p = llama3_llm.apply_model_template(instr, suffix)
r = llama3_llm.query_llm(p)
print(r)

{'text': '\nresponse: The maximum static pressure for the product EMED-3135.01-067 is 61.4377 kPa. This value is based on data from the year 2023, between January 1st and December 31st.'}


In [8]:
# Actualizamos la db con la nueva respuesta
from src.db.mongo.chats_manager import save_to_chat

save_to_chat(chat_document_copy)

Save to chat successfully
Conexion finalizada
